In [64]:
import torch
a=torch.tensor([1,3,5,600,5,3.0])
b=torch.tensor([2,5,3,8,7,6.0])
data=torch.stack([a,b],dim=1)              
norm_data=torch.nn.functional.normalize(data,dim=0)  #按列进行标准化
print(data)
print(norm_data)

tensor([[  1.,   2.],
        [  3.,   5.],
        [  5.,   3.],
        [600.,   8.],
        [  5.,   7.],
        [  3.,   6.]])
tensor([[0.0017, 0.1463],
        [0.0050, 0.3656],
        [0.0083, 0.2194],
        [0.9999, 0.5850],
        [0.0083, 0.5119],
        [0.0050, 0.4388]])


In [65]:
#从dataframe中加载数据
import numpy as np 
import os
import pandas as pd
#data=np.random.randn(5000,10)
#data=pd.DataFrame(data)
#data.to_csv(os.path.join(os.getcwd(),'data\\test_data.csv'))
file_path=os.path.join(os.getcwd(),'data\\test_data.csv')
data=pd.read_csv(file_path)
tensor=torch.Tensor(data.values)
length=len(tensor)
labels=torch.Tensor(list(range(length)))
labels=labels.unsqueeze(1)
torch_cat=torch.cat((tensor,labels),1)
train_tensor,test_tensor=torch.utils.data.random_split(tensor,[4000,1000])
train=torch.utils.data.DataLoader(train_tensor,batch_size=2,shuffle=True)   #从tensor矩阵中随机提取数据
next(iter(train))

tensor([[ 4.4490e+03, -1.7196e+00, -9.3505e-01,  4.6770e-01,  3.4054e-01,
         -1.5613e+00,  7.4920e-01, -2.8211e-01, -8.6673e-01, -1.5218e+00,
         -3.0984e-01],
        [ 1.4790e+03, -9.5651e-01,  4.6661e-02, -4.2733e-01,  1.0487e+00,
         -8.6718e-01, -4.7173e-02,  2.3986e+00,  2.1287e-01,  1.5464e-01,
         -7.6331e-01]])

In [8]:
'''
自定义dataset类，提取dataframe中的数据，__init__,__len__,__getitem__三个函数必须有
可参考pytorch官方文档，自定义dataset.
此方法可借用dataloader函数的能力，随机提取datas和labels.
'''
import os
import pandas as pd
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor #把Converts a PIL Image or numpy.ndarray (H x W x C) to a torch.FloatTensor of shape (C x H x W) in the range [0.0, 1.0]
class CustomDataset(Dataset):
    def __init__(self, dataset_path,labels):
        self.data_set = pd.read_csv(dataset_path)
        self.labels=labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        data=self.data_set
        data=torch.Tensor(data.values)
        data=data[idx]
        label =self.labels[idx]
        return data, label
labels=torch.Tensor(list(range(length)))
dt=CustomDataset(file_path,labels)    #实例化CustomDataset类，类的实例化传入初始函数中的参数
train=torch.utils.data.DataLoader(dt,batch_size=2,shuffle=True)   #直接将类的实例化对象传入即可
data_set,label=next(iter(train))
print(data_set,label)

tensor([[ 2.6660e+03, -1.0031e+00,  3.9287e-01, -6.2941e-01, -8.0789e-01,
          4.6374e-01, -5.5712e-02,  1.0213e+00, -3.2576e-01, -1.6442e-01,
          4.9701e-01],
        [ 8.4300e+02, -5.6125e-01, -1.1955e+00,  8.3157e-01, -2.5472e-01,
         -8.1902e-01,  3.7078e-01, -3.3378e-01,  1.0817e-01, -2.5823e-01,
         -1.5253e+00]]) tensor([2666.,  843.])


In [47]:
import torch
from torch import nn


class FullyConnectedModule(nn.Module):
    def __init__(self, input_size: int, output_size: int, hidden_size: int, n_hidden_layers: int):
        super().__init__()

        # input layer
        module_list = [nn.Linear(input_size, hidden_size), nn.ReLU()]
        # hidden layers
        for _ in range(n_hidden_layers):
            module_list.extend([nn.Linear(hidden_size, hidden_size), nn.ReLU()])
        # output layer
        module_list.append(nn.Linear(hidden_size, output_size))

        self.sequential = nn.Sequential(*module_list)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x of shape: batch_size x n_timesteps_in
        # output of shape batch_size x n_timesteps_out
        return self.sequential(x)

# test that network works as intended
network = FullyConnectedModule(input_size=5, output_size=2, hidden_size=10, n_hidden_layers=2)
x = torch.rand(20, 5)
network(x).shape



torch.Size([20, 2])

In [48]:
from typing import Dict

from pytorch_forecasting.models import BaseModel


class FullyConnectedModel(BaseModel):
    def __init__(self, input_size: int, output_size: int, hidden_size: int, n_hidden_layers: int, **kwargs):
        # saves arguments in signature to `.hparams` attribute, mandatory call - do not skip this
        self.save_hyperparameters()
        # pass additional arguments to BaseModel.__init__, mandatory call - do not skip this
        super().__init__(**kwargs)
        self.network = FullyConnectedModule(
            input_size=self.hparams.input_size,
            output_size=self.hparams.output_size,
            hidden_size=self.hparams.hidden_size,
            n_hidden_layers=self.hparams.n_hidden_layers,
        )

    def forward(self, x: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
        # x is a batch generated based on the TimeSeriesDataset
        network_input = x["encoder_cont"].squeeze(-1)
        prediction = self.network(network_input)

        # rescale predictions into target space
        prediction = self.transform_output(prediction, target_scale=x["target_scale"])

        # We need to return a dictionary that at least contains the prediction
        # The parameter can be directly forwarded from the input.
        # The conversion to a named tuple can be directly achieved with the `to_network_output` function.
        return self.to_network_output(prediction=prediction)

In [52]:
import numpy as np
import pandas as pd

test_data = pd.DataFrame(
    dict(
        value=np.random.rand(30) - 0.5,
        group=np.repeat(np.arange(3), 10),
        time_idx=np.tile(np.arange(10), 3),
    )
)
test_data

,value,group,time_idx
0,0.496879,0,0
1,-0.114598,0,1
2,-0.326793,0,2
3,0.207059,0,3
4,-0.099711,0,4
5,0.355300,0,5
6,-0.320841,0,6
7,-0.189331,0,7
8,0.331821,0,8
9,-0.323094,0,9


In [53]:
from pytorch_forecasting import TimeSeriesDataSet

# create the dataset from the pandas dataframe
dataset = TimeSeriesDataSet(
    test_data,
    group_ids=["group"],
    target="value",
    time_idx="time_idx",
    min_encoder_length=5,
    max_encoder_length=5,
    min_prediction_length=2,
    max_prediction_length=2,
    time_varying_unknown_reals=["value"],
)

In [57]:
# convert the dataset to a dataloader
dataloader = dataset.to_dataloader(batch_size=2)

# and load the first batch
x, y = next(iter(dataloader))
print("x =", x)
print("\ny =", y)
print("\nsizes of x =")
for key, value in x.items():
    print(f"\t{key} = {value.size()}")

x = {'encoder_cat': tensor([], size=(2, 5, 0), dtype=torch.int64), 'encoder_cont': tensor([[[-0.9283],
         [-0.5198],
         [ 1.7849],
         [-0.4460],
         [ 0.3851]],

        [[ 0.7363],
         [-0.3819],
         [ 1.2766],
         [-1.1878],
         [-0.7085]]]), 'encoder_target': tensor([[-0.2496, -0.1375,  0.4948, -0.1173,  0.1107],
        [ 0.2071, -0.0997,  0.3553, -0.3208, -0.1893]]), 'encoder_lengths': tensor([5, 5]), 'decoder_cat': tensor([], size=(2, 2, 0), dtype=torch.int64), 'decoder_cont': tensor([[[-1.1478],
         [ 1.1793]],

        [[ 1.1910],
         [-1.1961]]]), 'decoder_target': tensor([[-0.3098,  0.3286],
        [ 0.3318, -0.3231]]), 'decoder_lengths': tensor([2, 2]), 'decoder_time_idx': tensor([[5, 6],
        [8, 9]]), 'groups': tensor([[2],
        [0]]), 'target_scale': tensor([[0.0051, 0.2744],
        [0.0051, 0.2744]])}

y = (tensor([[-0.3098,  0.3286],
        [ 0.3318, -0.3231]]), None)

sizes of x =
	encoder_cat = torch.Size([

In [61]:
def forward(self, x: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
    # x is a batch generated based on the TimeSeriesDataset
    network_input = x["encoder_cont"].squeeze(-1)
    prediction = self.network(network_input)

    # rescale predictions into target space
    prediction = self.transform_output(prediction, target_scale=x["target_scale"])

    # We need to return a dictionary that at least contains the prediction
    # The parameter can be directly forwarded from the input.
    # The conversion to a named tuple can be directly achieved with the `to_network_output` function.
    return self.to_network_output(prediction=prediction)

In [62]:
model = FullyConnectedModel.from_dataset(dataset, input_size=5, output_size=2, hidden_size=10, n_hidden_layers=2)
x, y = next(iter(dataloader))
model(x)

C:\Python39\lib\site-packages\pytorch_lightning\utilities\parsing.py:244: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(
C:\Python39\lib\site-packages\pytorch_lightning\utilities\parsing.py:244: UserWarning: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
  rank_zero_warn(


Output(prediction=tensor([[ 0.0689, -0.0751],
        [ 0.0689, -0.0895]], grad_fn=<AddBackward0>))

In [63]:
dataset.x_to_index(x)

,time_idx,group
0,7,0
1,7,1
